In [1]:
import pandas as pd
import time
import numpy as np

In [2]:
SWurl = pd.read_csv('../data/url-IEEEData-dict2-subword.csv')

In [3]:
SWurl

,Unnamed: 0,word2,label,nurl
0,0,belajarprogramming com newpaypal loggy paypal...,1,"bela, jar, pro, gram, ming, com, new, pa, yp, ..."
1,1,ninedotlabs com att2 index.php,1,"nine, dot, lab, s, com, at, t, 2, index, ., php"
2,2,www almostfabulous com youker,0,"www, almost, fa, bu, lous, com, you, ker"
3,3,stbcomputer com autodesk components www.paypa...,1,"st, bc, omp, uter, com, auto, des, k, componen..."
4,4,members tripod com chang.ling,0,"members, trip, od, com, chang, ., ling"
...,...,...,...,...
95777,95777,www winehq com,0,"www, wine, h, q, com"
95778,95778,2933517 com js,1,"293, 35, 17, com, j, s"
95779,95779,imajinsmiles com Questions.html,1,"im, aj, ins, mi, les, com, questions, ., html"
95780,95780,thesecondhalfonline com wp-includes js tinymc...,1,"these, con, dha, lf, on, line, com, w, p, -, i..."


In [4]:
def add_word_to_dict(word,worddict):
    if (word not in worddict):
        worddict[word] = len(worddict)+1

In [5]:
worddict = dict()
for i in range(0,len(SWurl['nurl'])):
    a=SWurl['nurl'][i].split(', ')
    for j in range(0,len(a)):
        add_word_to_dict(a[j], worddict)

In [6]:
len(worddict)

14221

In [7]:
wordLength = 0
for i in range(0,len(SWurl['nurl'])):
    a=SWurl['nurl'][i].split(', ')
    if (len(a) > wordLength):
        wordLength = len(a)

In [8]:
wordLength

329

In [9]:
X = np.zeros(shape=(len(SWurl['nurl']),wordLength))
for i in range(0, len(SWurl['nurl'])):
    a = SWurl['nurl'][i].split(', ')
    for j in range(0,len(a)):
        X[i, j] = worddict.get(a[j])

In [10]:
SWX = pd.DataFrame(X)
SWX = SWX.astype(int)
SWX

,0,1,2,3,4,5,6,7,8,9,...,319,320,321,322,323,324,325,326,327,328
0,1,2,3,4,5,6,7,8,9,10,...,0,0,0,0,0,0,0,0,0,0
1,18,19,20,21,6,22,23,24,25,15,...,0,0,0,0,0,0,0,0,0,0
2,27,28,29,30,31,6,32,33,0,0,...,0,0,0,0,0,0,0,0,0,0
3,34,35,36,37,6,38,39,40,41,27,...,0,0,0,0,0,0,0,0,0,0
4,68,69,70,6,71,15,72,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95777,27,6117,16,588,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95778,9549,452,240,6,79,21,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95779,84,3979,568,102,512,6,2850,15,101,0,...,0,0,0,0,0,0,0,0,0,0
95780,4655,668,8283,1696,322,913,6,172,199,45,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X = SWX
X.shape

(95782, 329)

In [12]:
Y = SWurl['label']

In [13]:
Y.shape

(95782,)

In [14]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2, random_state=42)

In [15]:
Xtrain.shape, len(ytrain)

((76625, 329), 76625)

In [16]:
Xtest.shape, len(ytest)

((19157, 329), 19157)

In [17]:
from keras.layers import Embedding
from keras.models import Sequential, Model
from keras.layers import Dense, Activation
from keras.layers import Flatten, Conv1D, SpatialDropout1D, MaxPooling1D,AveragePooling1D, merge, concatenate, Input, Dropout
from keras.utils import plot_model
from keras.layers import GaussianNoise
from keras.regularizers import l1, l2, l1_l2
from keras.layers import SpatialDropout1D, SpatialDropout2D, SpatialDropout3D
from keras.callbacks import EarlyStopping

In [18]:
vocab_size = len(worddict)
maxLength = wordLength

In [19]:
vocab_size = 14222
maxLength = 329

In [20]:
vocab_size, maxLength

(14222, 329)

In [21]:
def model(max_length, vocab_size, output_dim, filter_sizes, num_filters, dropout_rate, dense_units):
    embed_input = Input(shape=(max_length,))
    x = Embedding(vocab_size, output_dim, input_length=max_length)(embed_input)
    
    noise = GaussianNoise(0.5)(x)
    
    drop = SpatialDropout1D(dropout_rate)(noise)
    pooled_outputs = []
    for i in range(len(filter_sizes)):
        conv = Conv1D(num_filters, kernel_size=filter_sizes[i], padding='same', activation='relu')(drop)
        conv = Flatten()(conv)           
        pooled_outputs.append(conv)
        
    
    flat = Flatten()(noise)
    pooled_outputs.append(flat)
    
    merge = concatenate(pooled_outputs)
    drop = Dropout(dropout_rate)(merge)
    
    dense_1 = Dense(dense_units, activation='relu')(drop)
    dense_2 = Dense(dense_units, activation='relu')(dense_1)
    dense_3 = Dense(dense_units, activation='relu')(dense_2)
    out = Dense(1, activation='sigmoid')(dense_3)
    
    model = Model(inputs=embed_input, outputs=out)
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    print(model.summary())
    return model

import time
start=time.time()
model = model(maxLength, vocab_size, 48, [2, 3, 4, 5, 10], 256, 0.4, 128)
history = model.fit(Xtrain, ytrain, validation_data=(Xtest, ytest), epochs=10, verbose=1) 

print('time for training the model = {:.0f} sec'.format(time.time()-start))
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
print('Test Loss: %f' % (loss*100))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 329)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 329, 48)      682656      input_1[0][0]                    
__________________________________________________________________________________________________
gaussian_noise (GaussianNoise)  (None, 329, 48)      0           embedding[0][0]                  
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 329, 48)      0           gaussian_noise[0][0]             
______________________________________________________________________________________________

In [22]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(Xtest)
print(confusion_matrix(ytest, np.rint(y_pred)))

[[9407  180]
 [ 351 9219]]


In [23]:
model.save("../saved_models/Subword-CNN_iter10.h5")

In [24]:
history2 = model.fit(Xtrain, ytrain, validation_data=(Xtest, ytest), epochs=10, verbose=1) 

print('time for training the model = {:.0f} sec'.format(time.time()-start))
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
print('Test Loss: %f' % (loss*100))

Epoch 1/10
2395/2395 [==============================] - 1204s 503ms/step - loss: 0.0596 - accuracy: 0.9779 - val_loss: 0.0797 - val_accuracy: 0.9708
Epoch 2/10
2395/2395 [==============================] - 1181s 493ms/step - loss: 0.0535 - accuracy: 0.9804 - val_loss: 0.0799 - val_accuracy: 0.9726
Epoch 3/10
2395/2395 [==============================] - 1201s 502ms/step - loss: 0.0505 - accuracy: 0.9815 - val_loss: 0.0747 - val_accuracy: 0.9731
Epoch 4/10
2395/2395 [==============================] - 1176s 491ms/step - loss: 0.0480 - accuracy: 0.9823 - val_loss: 0.0736 - val_accuracy: 0.9749
Epoch 5/10
2395/2395 [==============================] - 1183s 494ms/step - loss: 0.0434 - accuracy: 0.9843 - val_loss: 0.0741 - val_accuracy: 0.9739
Epoch 6/10
2395/2395 [==============================] - 1176s 491ms/step - loss: 0.0409 - accuracy: 0.9854 - val_loss: 0.0777 - val_accuracy: 0.9742
Epoch 7/10
2395/2395 [==============================] - 1172s 489ms/step - loss: 0.0391 - accuracy: 0.9863

In [25]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(Xtest)
print(confusion_matrix(ytest, np.rint(y_pred)))

[[9424  163]
 [ 296 9274]]


In [26]:
model.save("../saved_models/Subword-CNN_iter20.h5")